In [ ]:
import pandas as pd
import numpy as np
import json
import struct
from os import system

1\. **Text files**

Perform the following operations on plain `txt` files:

+ create a list of integrer numbers and then save it to a text file named `data_int.txt`. Run the `cat` command to print the content of the file.
+ create a matrix of 5x5 floats and then save it to a text file named `data_float.txt`. Use the `cat` command to print the content of the file.
+ load the `txt` file of the previous point and convert it to a `csv` file by hand.

In [ ]:
integers_list = range(1,10)

output_file = "data_int.txt"

with open(output_file, 'w') as out_file:
  for line in integers_list:
    out_file.write(str(line) + '\n') 
# i use windows, so i should use "!type" except for "!cat"
!type data_int.txt
print('---------------------------------------------------------')
#---------------------------------------------------------------
matrix = np.random.random(size=(5,5))

output_file = "data_float.txt"

with open(output_file, 'w') as out_file:
  for line in matrix:
    out_file.write(str(line) + '\n') 

!type data_float.txt
print('---------------------------------------------------------')
#---------------------------------------------------------------
df = pd.read_fwf('data_float.txt')
df.to_csv('data_float.csv')

2\. **JSON files**

Load the file *user_data.json*, which can be found at:

- https://www.dropbox.com/s/sz5klcdpckc39hd/user_data.json

and filter the data by the "CreditCardType" when it equals to "American Express". Than save the data to a new CSV file.

In [ ]:
result = []

data = json.load(open('C:/Users/sina tavakoli/Desktop/user_data.json'))
# print(data)
for parsedData in data:
    if (parsedData['CreditCardType'] == 'American Express'):
        result.append(parsedData)

df = pd.DataFrame(data=result)
df.to_csv("resulted_Data.csv")

3\. **CSV files with Pandas**

Load the file from this url:

- https://www.dropbox.com/s/kgshemfgk22iy79/mushrooms_categorized.csv

with Pandas. 

+ explore and print the DataFrame
+ calculate, using `groupby()`, the average value of each feature, separately for each class
+ save the file in a JSON format.

In [ ]:
data  = pd.read_csv('C:/Users/sina tavakoli/Desktop/mushrooms_categorized.csv')
avg_data = data.groupby(['class']).mean()
avg_data.to_json("mushrooms_categorized.json", orient="columns")
print(data)
avg_data

4\. **Reading the credit card numbers**

Get the binary file named *credit_card.dat* from this address:

- https://www.dropbox.com/s/8m0syw2tkul3dap/credit_card.dat

and convert the data into the real credit card number, knowing that:
- each line corresponds to a credit card number, which consists of 16 characters (which are numbers in the 0-9 range) divided in 4 blocks, with a whitespace between each block
- each character is written using a 6 bit binary representation (including the whitespace)
- the final 4 bits of each line are a padding used to determine the end of the line, and can be ignored

*Hint*: convert the binary numbers to the decimal representation first, and then use the `chr()` function to convert the latter to a char

In [ ]:
master_file = "C:/Users/sina tavakoli/Desktop/credit_card.dat"
with open(master_file, 'r') as out_file:
    for line in out_file:
        print(''.join([chr(int(line[i:i+6],2)) for i in range(0, len(line), 6)]))

5\. **Write data to a binary file**

a) Start from the `data/data_000637.txt` file that we have used during the previous lectures, and convert it to a binary file according to the format defined below:

In [ ]:
from IPython.display import Image
Image("images/data_format.png")

*Hints*:
- Read the first 10 lines using Pandas
- Iterate over the DataFrame rows
- For every row, ``pack'' the values (features) into a single 64-bit word, according to the format specified above. Use bit-wise shifts and operators to do so.
- Write each 64-bit word to a binary file. You can use `struct` in this way:
```
binary_file.write( struct.pack('<q', word) )
```
where `word` is the 64-bit word.
- Close the file after completing the loop.

b) Check that the binary file is correctly written by reading it with the code used in the lecture `06_OSEMN.ipynb`, and verify that the content of the `txt` and binary files is consistent.

c) What is the difference of the size on disk between equivalent `txt` and binary files?

In [ ]:
import struct
master_data = pd.read_csv("C:/Users/sina tavakoli/Desktop/data_000637.txt", nrows = 10)
print(master_data)

data = pd.read_csv("C:/Users/sina tavakoli/Desktop/data_000637.txt")

with open("binary_file.dat", "wb") as binary_file:
    for i in range(len(data["HEAD"])):
        
        x1 = 1 << 62
        fpga = data.loc[data.index[i],"FPGA"] << 58
        tdcc = data.loc[data.index[i],"TDC_CHANNEL"] << 49
        orbit = data.loc[data.index[i],"ORBIT_CNT"] << 17
        bx = data.loc[data.index[i],"BX_COUNTER"] << 5
        tdcm = data.loc[data.index[i],"TDC_MEAS"]
        wordr = x1|fpga|tdcc|orbit|bx|tdcm
        wordx = x1^fpga^tdcc^orbit^bx^tdcm        
        aux1 = int(wordr & 0xFFFFFFFF)
        aux2 = int((wordr >> 32) & 0xFFFFFFFF)
        binary_file.write(struct.pack("<q",aux1))
binary_file.close()    